In [142]:
# Preprocessing
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder,
    LabelEncoder,
    OrdinalEncoder,
    StandardScaler,
    FunctionTransformer,
    PolynomialFeatures,
)
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV

# from workalendar.europe import France

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor, Pool
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from timeit import default_timer as timer

plt.style.use("fivethirtyeight")

path = Path("mdsb-2023")

In [143]:
def train_test_split_temporal(X, y, delta_threshold="60 days"):

    cutoff_date = X["date"].max() - pd.Timedelta(delta_threshold)
    mask = X["date"] <= cutoff_date
    X_train, X_valid = X.loc[mask], X.loc[~mask]
    y_train, y_valid = y[mask], y[~mask]

    return X_train, y_train, X_valid, y_valid

In [144]:
def add_lags(X, cols_to_lag=["t", "u", "vv", "nnuage4"], lag_list=[2, -24, -2]):
    X = X.copy()

    feature_columns = [col for col in X.columns if col in cols_to_lag]

    for l in lag_list:
        lag_columns = [f"{col}_lag{l}" for col in feature_columns]
        X[lag_columns] = X[feature_columns].shift(periods=l, axis=0)
        X[lag_columns] = (
            X[lag_columns]
            .interpolate(method="linear")
            .interpolate(method="bfill")
            .interpolate(method="ffill")
        )

    return X


def add_moving_average(
    X, cols_to_ma=["t", "u", "vv", "nnuage4"], window_list=[24 * 7, 24], centered=True
):
    X = X.copy()

    feature_columns = [col for col in X.columns if col in cols_to_ma]

    for w in window_list:
        ma_columns = [f"{col}_ma{w}" for col in feature_columns]
        X[ma_columns] = X[feature_columns].rolling(window=w, center=centered).mean()
        X[ma_columns] = (
            X[ma_columns]
            .interpolate(method="linear")
            .interpolate(method="bfill")
            .interpolate(method="ffill")
        )

    return X

### Define pipeline functions

In [145]:
def _encode_dates(X, col_name="date"):
    X = X.copy()

    #     X["year"] = X[col_name].dt.year
    X["month"] = X[col_name].dt.month
    #     X["day"] = X[col_name].dt.day
    X["weekday"] = X[col_name].dt.weekday
    X["hour"] = X[col_name].dt.hour
    #     X['isweekend'] = X[col_name].apply(lambda x: 0 if x.weekday() in range(5) else 1)

    #     cal = France()
    #     X['is_holiday'] = X['date'].apply(cal.is_holiday)
    #     X['working_day'] = X['date'].apply(cal.is_working_day)

    X["month_sin"] = np.sin(2 * np.pi * X["date"].dt.month / 12)
    X["month_cos"] = np.cos(2 * np.pi * X["date"].dt.month / 12)

    X["day_sin"] = np.sin(2 * np.pi * X["date"].dt.day / X["date"].dt.days_in_month)
    X["day_cos"] = np.cos(2 * np.pi * X["date"].dt.day / X["date"].dt.days_in_month)

    X["hour_sin"] = np.sin(2 * np.pi * X["date"].dt.hour / 24)
    X["hour_cos"] = np.cos(2 * np.pi * X["date"].dt.hour / 24)

    X[["month", "weekday", "hour"]] = X[["month", "weekday", "hour"]].astype("category")

    return X.drop(columns=[col_name])

In [146]:
def _encode_covid(X, col_name="date"):
    X = X.copy()

    # Create masks for lockdown dates
    lockdown_1 = (X["date"] >= "2020-10-17") & (
        X["date"] <= "2020-12-14"
    )  # & ((X['date'].dt.hour >= 21) | (X['date'].dt.hour <= 6))

    lockdown_2 = (X["date"] >= "2020-12-15") & (
        X["date"] <= "2021-02-26"
    )  # & ((X['date'].dt.hour >= 18) | (X['date'].dt.hour <= 6))

    lockdown_3 = (X["date"] >= "2021-02-27") & (
        X["date"] <= "2021-05-02"
    )  # & ((X['date'].dt.hour >= 19) | (X['date'].dt.hour <= 6))

    X["Covid"] = 0
    X.loc[lockdown_1 | lockdown_2 | lockdown_3, "Covid"] = 1

    return X

In [147]:
def _merge_external_data(X, include_lags=True, include_ma=True):
    to_keep = [
        "date",
        "hnuage4",
        "t",
        "ctype4",
        "u",
        "etat_sol",
        "perssfrai",
        "tx12",
        "cm",
        "tn12",
        "tend24",
        "vv",
        "rafper",
        "rr24",
        "td",
        "rr3",
        "hnuage3",
        "hnuage1",
    ]

    ext_data = pd.read_csv(path / "external_data.csv", parse_dates=["date"])[to_keep]

    ext_data.drop(columns=ext_data.columns[ext_data.isna().sum() > 1000])

    full_date_range = pd.date_range(
        start=np.min([np.min(data.date), np.min(test.date)]),
        end=np.max([np.max(data.date), np.max(test.date)]),
        freq="H",
    )
    full_date_range = pd.DataFrame({"date": full_date_range})

    ext_data = full_date_range.merge(ext_data, on="date", how="left")

    columns_to_interpolate = ext_data.drop(columns="date").columns
    ext_data[columns_to_interpolate] = (
        ext_data[columns_to_interpolate]
        .interpolate(method="polynomial", order=3)
        .interpolate(method="bfill")
        .interpolate(method="ffill")
    )

    if include_lags:
        ext_data = add_lags(ext_data)

    if include_ma:
        ext_data = add_moving_average(ext_data)

    to_drop = [
        "vv_lag2",
        "t_lag-24",
        "u_lag-24",
        "vv_lag-24",
        "tx12",
        "etat_sol",
        "vv_lag-2",
        "u",
        "ctype4",
        "t",
        "u_lag2",
        "vv_ma24",
        "t_lag2",
        "tend24",
        "u_ma24",
        "vv_ma168",
        "cm",
        "hnuage1",
        "hnuage3",
        "rr3",
        "td",
        "rr24",
        "rafper",
        "vv",
        "u_ma168",
        "hnuage4",
    ]

    ext_data.drop(columns=to_drop, inplace=True)

    X = X.copy()

    X["date"] = X["date"].astype("datetime64[ns]")
    ext_data["date"] = ext_data["date"].astype("datetime64[ns]")

    X["orig_index"] = np.arange(X.shape[0])

    X = pd.merge_asof(X.sort_values("date"), ext_data.sort_values("date"), on="date")

    # Sort back to the original order
    X = X.sort_values("orig_index")
    del X["orig_index"]

    return X

In [148]:
def _gas_price_encoder(X):
    X = X.copy()
    X["gas_price"] = 1

    gas_prices = np.array(
        [
            1.22,
            1.21,
            1.22,
            1.27,
            1.31,
            1.36,
            1.4,
            1.39,
            1.4,
            1.43,
            1.45,
            1.45,
            1.46,
            1.56,
        ]
    )

    years = [
        2020,
        2020,
        2020,
        2020,
        2021,
        2021,
        2021,
        2021,
        2021,
        2021,
        2021,
        2021,
        2021,
        2021,
    ]

    months = [9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

    for i, price in enumerate(gas_prices):
        X.loc[
            (X.date.dt.month == months[i]) & (X.date.dt.year == years[i]), "gas_price"
        ] = price

    return X

# Modeling functions

In [63]:
from prophet import Prophet


def prophet_fit_pred(data, group, cut=pd.Timedelta("60 days"), plot_components=False):
    data = data.copy()
    data["floor"] = 0

    data = data[
        data["group"] == group
    ]  # To test on a single group only, for simplicity
    data = data[["ds", "y"]]

    cutoff_date = data["ds"].max() - cut
    mask = data["ds"] <= cutoff_date
    train_prophet, test_prophet = data.loc[mask], data.loc[~mask]

    # Initialize and fit the model
    prophet = Prophet(
        interval_width=0.95,
        growth="linear",
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
        seasonality_mode="multiplicative",
    )
    prophet.add_country_holidays(country_name="FR")

    prophet.fit(train_prophet)

    future = prophet.make_future_dataframe(
        periods=cut.days * 24, freq="h", include_history=True
    )

    # predict over the dataset
    components = prophet.predict(future)

    pred = components.iloc[-60 * 24 :, [0, -1]]  # Only keep the predicted

    if plot_components:
        prophet.plot_components(components)

    return pred, test_prophet, components

## Import main dataset

In [149]:
data = pd.read_parquet(path / "train.parquet")
test = pd.read_parquet(path / "final_test.parquet")

targets = ["bike_count", "log_bike_count"]

In [150]:
data.drop(
    columns=[
        "site_name",
        "counter_id",
        "site_id",
        "counter_installation_date",
        "coordinates",
        "counter_technical_id",
    ],
    inplace=True,
)
test.drop(
    columns=[
        "site_name",
        "counter_id",
        "site_id",
        "counter_installation_date",
        "coordinates",
        "counter_technical_id",
    ],
    inplace=True,
)

## Prophet

In [71]:
y_hat = pd.DataFrame(columns=["ds", "yhat", "group"])
y_true = pd.DataFrame(columns=["ds", "y", "group"])

forecast = pd.DataFrame()

for i, v in enumerate(data["counter_name"].unique()[:]):
    print(f"\n==[ Start group [{i}/{data['counter_name'].nunique()}] ]==")
    data_prophet = data.rename(
        columns={"date": "ds", "log_bike_count": "y", "counter_name": "group"}
    ).sort_values(["group", "ds"])
    pred, test, components = prophet_fit_pred(data_prophet, v)
    components["group"] = v
    pred["group"] = v
    test["group"] = v

    forecast = pd.concat([forecast, components])

    # Saving in the full datasets
    y_hat = pd.concat([y_hat, pred])
    y_true = pd.concat([y_true, test])

23:11:42 - cmdstanpy - INFO - Chain [1] start processing
23:11:43 - cmdstanpy - INFO - Chain [1] done processing
23:11:47 - cmdstanpy - INFO - Chain [1] start processing
23:11:48 - cmdstanpy - INFO - Chain [1] done processing
23:11:52 - cmdstanpy - INFO - Chain [1] start processing
23:11:53 - cmdstanpy - INFO - Chain [1] done processing
23:11:56 - cmdstanpy - INFO - Chain [1] start processing
23:11:57 - cmdstanpy - INFO - Chain [1] done processing
23:12:01 - cmdstanpy - INFO - Chain [1] start processing
23:12:02 - cmdstanpy - INFO - Chain [1] done processing
23:12:06 - cmdstanpy - INFO - Chain [1] start processing
23:12:07 - cmdstanpy - INFO - Chain [1] done processing
23:12:10 - cmdstanpy - INFO - Chain [1] start processing
23:12:11 - cmdstanpy - INFO - Chain [1] done processing
23:12:14 - cmdstanpy - INFO - Chain [1] start processing
23:12:15 - cmdstanpy - INFO - Chain [1] done processing
23:12:19 - cmdstanpy - INFO - Chain [1] start processing
23:12:20 - cmdstanpy - INFO - Chain [1]

In [131]:
import regex as re

prophet_fit = forecast
prophet_fit.rename(columns=lambda c: re.sub(r"'", "", c), inplace=True)
prophet_fit.rename(columns=lambda c: re.sub(r"\s+", "", c), inplace=True)

selected_cols = [
    "ds",
    "ChristmasDay",
    "NewYearsDay",
    "daily",
    "weekly",
    "yhat",
    "group",
]

prophet_fit = prophet_fit[selected_cols]
prophet_fit = prophet_fit.rename(
    columns={"ds": "date", "group": "counter_name"}
).sort_values(
    by=["counter_name", "date"]
)  # This can be merged to main dataset for extra features for the ml model

In [132]:
def _merge_prophet(X):
    X = X.copy()

    X["orig_index"] = np.arange(X.shape[0])
    X = pd.merge(
        X.sort_values("date"),
        prophet_fit.sort_values("date"),
        on=["date", "counter_name"],
    )

    # Sort back to the original order
    X = X.sort_values("orig_index")
    del X["orig_index"]
    return X

## Model

In [151]:
X, y = data.drop(columns=targets), data["log_bike_count"]

In [152]:
X_train, y_train, X_test, y_test = train_test_split_temporal(X, y)

print(
    f'Train: n_samples={X_train.shape[0]},  {X_train["date"].min()} to {X_train["date"].max()}'
)
print(
    f'Test: n_samples={X_test.shape[0]},  {X_test["date"].min()} to {X_test["date"].max()}'
)

Train: n_samples=416187,  2020-09-01 01:00:00 to 2021-07-11 23:00:00
Test: n_samples=80640,  2021-07-12 00:00:00 to 2021-09-09 23:00:00


In [153]:
target_name = "log_bike_count"

prophet_merger = FunctionTransformer(_merge_prophet, validate=False)
data_merger = FunctionTransformer(_merge_external_data, validate=False)
covid_encoder = FunctionTransformer(_encode_covid, validate=False)
gas_encoder = FunctionTransformer(_gas_price_encoder, validate=False)
date_encoder = FunctionTransformer(_encode_dates, validate=False)

date_cols = (
    _encode_dates(X_train[["date"]]).select_dtypes(include="category").columns.tolist()
)
categorical_cols = ["counter_name"] + date_cols

In [154]:
# {'learning_rate': 0.16263414906434522, 'n_estimators': 633}
best_params = {
    "learning_rate": 0.16,
    "max_depth": 8,
    "n_estimators": 630,
    "subsample": 0.8,
    "od_pval": 1e-5,
}

regressor = CatBoostRegressor(**best_params)

pipe = Pipeline(
    [
        ("merge external", data_merger),
        ("gas prices encoder", gas_encoder),
        ("covid encoder", covid_encoder),
        ("date encoder", date_encoder),
        ("regressor", regressor),
    ]
)

In [155]:
val_pool = Pool(
    _encode_dates(_encode_covid(_gas_price_encoder(_merge_external_data(X_test)))),
    label=y_test,
    cat_features=categorical_cols,
)
pipe.fit(
    X_train,
    y_train,
    regressor__cat_features=categorical_cols,
    regressor__early_stopping_rounds=70,
    regressor__eval_set=val_pool,
)

0:	learn: 1.5002806	test: 1.2899522	best: 1.2899522 (0)	total: 471ms	remaining: 4m 56s
1:	learn: 1.3410761	test: 1.1430619	best: 1.1430619 (1)	total: 826ms	remaining: 4m 19s
2:	learn: 1.2114159	test: 1.0249488	best: 1.0249488 (2)	total: 1.18s	remaining: 4m 6s
3:	learn: 1.1070993	test: 0.9328724	best: 0.9328724 (3)	total: 1.45s	remaining: 3m 47s
4:	learn: 1.0216898	test: 0.8588225	best: 0.8588225 (4)	total: 1.76s	remaining: 3m 39s
5:	learn: 0.9542367	test: 0.8058077	best: 0.8058077 (5)	total: 2.04s	remaining: 3m 32s
6:	learn: 0.8894080	test: 0.7747558	best: 0.7747558 (6)	total: 2.48s	remaining: 3m 40s
7:	learn: 0.8446885	test: 0.7450361	best: 0.7450361 (7)	total: 2.81s	remaining: 3m 38s
8:	learn: 0.8092048	test: 0.7224593	best: 0.7224593 (8)	total: 3.09s	remaining: 3m 33s
9:	learn: 0.7721090	test: 0.7231426	best: 0.7224593 (8)	total: 3.45s	remaining: 3m 34s
10:	learn: 0.7488907	test: 0.7090368	best: 0.7090368 (10)	total: 3.74s	remaining: 3m 30s
11:	learn: 0.7208418	test: 0.7048527	best:

93:	learn: 0.4600892	test: 0.6554377	best: 0.6554377 (93)	total: 32.5s	remaining: 3m 5s
94:	learn: 0.4596520	test: 0.6562645	best: 0.6554377 (93)	total: 32.9s	remaining: 3m 5s
95:	learn: 0.4590148	test: 0.6555258	best: 0.6554377 (93)	total: 33.3s	remaining: 3m 5s
96:	learn: 0.4581310	test: 0.6552049	best: 0.6552049 (96)	total: 33.7s	remaining: 3m 5s
97:	learn: 0.4569701	test: 0.6555532	best: 0.6552049 (96)	total: 34.1s	remaining: 3m 5s
98:	learn: 0.4563756	test: 0.6551844	best: 0.6551844 (98)	total: 34.6s	remaining: 3m 5s
99:	learn: 0.4554170	test: 0.6546830	best: 0.6546830 (99)	total: 35.1s	remaining: 3m 5s
100:	learn: 0.4545341	test: 0.6548499	best: 0.6546830 (99)	total: 35.4s	remaining: 3m 5s
101:	learn: 0.4537132	test: 0.6546642	best: 0.6546642 (101)	total: 35.7s	remaining: 3m 4s
102:	learn: 0.4533175	test: 0.6547394	best: 0.6546642 (101)	total: 36.1s	remaining: 3m 4s
103:	learn: 0.4527920	test: 0.6548206	best: 0.6546642 (101)	total: 36.4s	remaining: 3m 4s
104:	learn: 0.4517609	tes

184:	learn: 0.4183458	test: 0.6505628	best: 0.6501862 (152)	total: 1m 5s	remaining: 2m 37s
185:	learn: 0.4178860	test: 0.6504953	best: 0.6501862 (152)	total: 1m 5s	remaining: 2m 37s
186:	learn: 0.4174988	test: 0.6511987	best: 0.6501862 (152)	total: 1m 6s	remaining: 2m 37s
187:	learn: 0.4170796	test: 0.6512199	best: 0.6501862 (152)	total: 1m 6s	remaining: 2m 37s
188:	learn: 0.4169388	test: 0.6511115	best: 0.6501862 (152)	total: 1m 7s	remaining: 2m 36s
189:	learn: 0.4166014	test: 0.6500204	best: 0.6500204 (189)	total: 1m 7s	remaining: 2m 36s
190:	learn: 0.4163665	test: 0.6499954	best: 0.6499954 (190)	total: 1m 8s	remaining: 2m 36s
191:	learn: 0.4161934	test: 0.6498395	best: 0.6498395 (191)	total: 1m 8s	remaining: 2m 36s
192:	learn: 0.4159733	test: 0.6499966	best: 0.6498395 (191)	total: 1m 8s	remaining: 2m 35s
193:	learn: 0.4156314	test: 0.6499179	best: 0.6498395 (191)	total: 1m 9s	remaining: 2m 35s
194:	learn: 0.4152697	test: 0.6498180	best: 0.6498180 (194)	total: 1m 9s	remaining: 2m 35s

Pipeline(steps=[('merge external',
                 FunctionTransformer(func=<function _merge_external_data at 0x0000025D0E72BD90>)),
                ('gas prices encoder',
                 FunctionTransformer(func=<function _gas_price_encoder at 0x0000025D5381AE60>)),
                ('covid encoder',
                 FunctionTransformer(func=<function _encode_covid at 0x0000025D0E72BA30>)),
                ('date encoder',
                 FunctionTransformer(func=<function _encode_dates at 0x0000025D06973250>)),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x0000025D53870580>)])

In [138]:
y_hat = pipe.predict(X_test)

y_hat = np.mean(y_hat, _merge_prophet(X_test)["yhat"])

print(
    f"Train set, RMSE={mean_squared_error(y_train, pipe.predict(X_train), squared=False):.2f}"
)
print(f"Test set, RMSE={mean_squared_error(y_test, y_hat, squared=False):.2f}")

Train set, RMSE=1.46
Test set, RMSE=1.44


# Submission

In [ ]:
pipe.fit(X, y)
prediction = pipe.predict(test)
prediction[prediction < 0] = 0

In [ ]:
submission = pd.DataFrame({"log_bike_count": prediction})

# submission = pd.DataFrame({'Id' : submission.index, 'log_bike_count' : prediction})
submission = pd.DataFrame({"Id": test.index, "log_bike_count": prediction})

submission.to_csv("submission.csv", index=False)